<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам

Цель проекта - подобрать тип и параметры модели для определения цены автомобиля по характеристикам, указанным в объявлении.

Задачи:
1. Собрать актуальные объявления с сайта auto.ru
2. Обработать тренировочный и тестовый датасеты перед их объединением.
3. Поработать с дубликатами, пропусками.
4. Провести EDA
5. Сделать Feature Engineering
6. Перебрать регресоры из разных библиотек и выбрать лучший.
7. Провести подбор оптимальных параметров моделей.
8. Применить бэггинг, стэкинг и ансамблирование моделей.

Для работы собраны данные с сайта auto.ru. В наборе данных следующие признаки:
* body_type - тип кузова,
* brand - марка автомобиля,
* car_url - url страницы объявления
* color - цвет автомобиля,
* engine_displacement - объём двигателя,
* engine_volume - мощность двигателя,
* fuel_type - тип топлива,
* mileage - пробег,
* model_date - дата релиза модели,
* name - имя, введенное пользователем
* number_of_doors - количество дверей,
* production_date - дата производства автомобиля
* vehicle_configuration - конфигурация транспортного средства (ТС),
* vehicle_transmoission - тип коробки передач,
* owners_qty - количество владельцев,
* licence - паспорт ТС,
* type_of_drive - тип привода,
* steering_wheel - сторона руля,
* customs - этап растаможки,
* price - цена автомобиля, целевой параметр,
* sample - индикатор принадлежности данных к тесту (0) и трейну (1),

Часть признаков оказалась малоинформативной и от их использования я отказался в ходе EDA и при подготовке модели.

# IMPORTS, DATA & SETUP

Авторский парсер, адаптированный под запуск в Google Colab - https://github.com/Segodnya/data-science-studies/tree/master/module_6/auto_ru_parser

In [ ]:
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sys
import gc

from datetime import datetime

from wordcloud import WordCloud, STOPWORDS 
import string
from nltk.util import ngrams
from collections import Counter

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
import xgboost as xgb

from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler


%matplotlib inline

# settings to display all columns
pd.set_option("display.max_columns", None)

print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt


def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))


def get_boxplot(df, col):
    # draw a boxplot
    fig, axes = plt.subplots(figsize = (14, 4))
    sns.boxplot(x='price_log', y=col, data=df, ax=axes)
    axes.set_title('Boxplot for ' + col)
    plt.show()


def getFeaturesInfo(df):
    # num of unique values, first 10 unique values, null values count, type
    df_agg = df.agg({'nunique', lambda s: s.unique()[:10]})\
        .append(pd.Series(df.isnull().sum(), name='null'))\
        .append(pd.Series(df.dtypes, name='dtype'))\
        .transpose()

    return df_agg


def getOwningMonths(string):
    # returns the number of months from string value
    line = string.split()
    owning_month = 0
    
    if len(line) > 2:
        owning_month += 12 * float(line[0])
        owning_month += float(line[3])
    else:
        if 'год' in line[1] or 'лет' in line[1]:
            owning_month += 12 * float(line[0])
        else:
            owning_month += float(line[0])

    return owning_month


def calc_delta_ouned(x):
    if x['year'] > 0:
        return (2020 - x['year'])*12 - x['month'] + 9
    else: return 0


VERSION    = 20
DIR_TRAIN  = '../input/autoru-moscow-2021-08/'
DIR_TRAIN_OLD  = '../input/parsing-all-moscow-auto-ru-09-09-2020/'
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20
RANDOM_SEED = 42
USD_TRAIN = 73.2781 # 31.08.2021
USD_TRAIN_OLD = 75.9645 # 09.09.2020
USD_TEST = 77.78 # 21.10.2020


train = pd.read_csv(DIR_TRAIN+'autoru_moscow_2021_08.csv')
# clean data after parsing
train.drop(columns=train.columns[0], axis=1, inplace=True)
train.drop(train[train['Состояние'] == 'Состояние'].index, inplace=True)

train_old = pd.read_csv(DIR_TRAIN_OLD+'all_auto_ru_09_09_2020.csv')
train_old['model_name'] = train_old['model']
train_old['vendor'] = 0
train_old.drop(['model', 'Комплектация', 'start_date', 'hidden'], axis=1, inplace=True)

test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

# rename columns with lower eng letters
for df in [train, train_old, test]:
    df.rename(columns={
    'Владельцы': 'owners', 
    'Владение': 'owning', 
    'ПТС': 'pts', 
    'Привод': 'drive', 
    'Руль': 'wheel', 
    'Состояние': 'status', 
    'Таможня': 'customs'
    }, inplace=True)

In [ ]:
train.head(3).T

In [ ]:
train.info()

In [ ]:
getFeaturesInfo(train)

In [ ]:
train_old.head(3).T

In [ ]:
train_old.info()

In [ ]:
getFeaturesInfo(train_old)

In [ ]:
test.head(3).T

In [ ]:
test.info()

In [ ]:
getFeaturesInfo(test)

In [ ]:
sample_submission.head()

parsing_unixtime - время, когда сделан парсинг страниц (можно указать в начале файла, а не в каждой строке). Например, тестовыая выборка сделана Wed Oct 21 2020)

### Drop Duplicates

In [ ]:
train.drop_duplicates(subset=['car_url'], inplace=True)
train_old.drop_duplicates(inplace=True)

### Drop NaN

In [ ]:
# удаляем столбцы без диффиренцироиванных признаков
non_unique_cols = ['status', 'priceCurrency', 'customs']
train.drop(columns=non_unique_cols, inplace=True)
train_old.drop(columns=['status', 'customs'], inplace=True)
test.drop(columns=non_unique_cols, inplace=True)

# создадим флаги по записям, для которых отсутствуют сведения
train['owning_empty'] = pd.isna(train['owning']).astype('uint8')
train_old['owning_empty'] = pd.isna(train_old['owning']).astype('uint8')
test['owning_empty'] = pd.isna(test['owning']).astype('uint8')
# fill nans
train['owning'].fillna('0 месяцев', inplace=True)
train_old['owning'].fillna('0 месяцев', inplace=True)
test['owning'].fillna('0 месяцев', inplace=True)

# Пропуск данных о ПТС заполним значением 'Оригинал'
train_old['pts'].fillna('Оригинал', inplace=True)
test['pts'].fillna('Оригинал', inplace=True)

# пропуски во внешнем датасете
train_old['description'].fillna('', inplace=True)

change_owners = [0, 65000, 145000, 1e6] # список км смены владельца
labels = [1, 2, 3]
train_old['mileage_owners'] = pd.cut(train_old['mileage'], bins=change_owners,
                                     labels=labels, include_lowest=True) # добавим вспомогательное поле
train_old['owners'] = train_old['owners'].fillna(train_old['mileage_owners']) # заполним пропуски
train_old.drop('mileage_owners', axis=1, inplace=True) # удалим вспомогательное поле

# drop незаполненные объявления (4 шт)
train.dropna(subset=['owners'], inplace=True)
# drop незаполненные объявления (1 шт)
train_old.dropna(subset=['drive'], inplace=True)


# drop объявления без цены
train.dropna(subset=['price'], inplace=True)
train_old.dropna(subset=['price'], inplace=True)


pd.DataFrame([train.isna().sum(), train_old.isna().sum(), test.isna().sum()], index=['train', 'train_old', 'test']).T

# PREPROCESSING & MERGING

Для работы с объединенным датасетом необходимо привести в соответствие значения признаков.

Обработку данных будем вести отдельно для train и test с последующим объединением перед процессом моделирования, поскольку для обучения использовались две выборки, собранные по-разному и в разное время.

In [ ]:
# bodyType
train['bodyType'] = train['bodyType'].apply(lambda x: str(x).lower().split()[0])
train['bodyType'] = train['bodyType'].apply(lambda x: str(x).split('-')[0])
train_old['bodyType'] = train_old['bodyType'].apply(lambda x: str(x).lower().split()[0])
train_old['bodyType'] = train_old['bodyType'].apply(lambda x: str(x).split('-')[0])
test['bodyType'] = test['bodyType'].apply(lambda x: str(x).lower().split()[0])
test['bodyType'] = test['bodyType'].apply(lambda x: str(x).split('-')[0])

bodyType_drop = [item for item in list(test['bodyType'].unique()) if item not in list(train['bodyType'].unique())]
train_old['bodyType'].replace(bodyType_drop, 'other', inplace=True)
test['bodyType'].replace(bodyType_drop, 'other', inplace=True)


# brand
brand_drop = [item for item in list(train_old['brand'].unique()) if item not in list(test['brand'].unique())]
train_old['brand'].replace(brand_drop, 'other', inplace=True)

# delete unsuitable brands
# train_old.drop(train_old.loc[train_old['brand']=='other'].index, inplace=True)

# Create a top brands list (more than 350 ads in Dataset)
# all_brands = train['brand'].value_counts().index
# top_brands_list = train['brand'].value_counts()[train['brand'].value_counts() >= 5].index.tolist()
# brands_drop = list(set(all_brands)-set(top_brands_list))
# train.loc[train['brand'].isin(brands_drop), 'brand'] = 'Other'


# color
dict_color = {'040001':'чёрный', 'EE1D19':'красный', '0000CC':'синий', 
              'CACECB':'серебристый', '007F00':'зелёный', 'FAFBFB':'белый', 
              '97948F':'серый', '22A0F8':'голубой', '660099':'пурпурный', 
              '200204':'коричневый', 'C49648':'бежевый', 'DEA522':'золотистый', 
              '4A2197':'фиолетовый', 'FFD600':'жёлтый', 'FF8649':'оранжевый', 
              'FFC0CB':'розовый'}

train_old['color'] = train_old['color'].map(dict_color)


# engineDisplacement
train['engineDisplacement'] = train['engineDisplacement'].apply(lambda x: str(x).replace('LTR', ''))
train['engineDisplacement'].replace(' ', 0.0, inplace=True)
train['engineDisplacement'] = train['engineDisplacement'].apply(lambda x: float(x))
train['engineDisplacement'].replace(0.0, train['engineDisplacement'].median(), inplace=True)

train_old['engineDisplacement'] = train_old['engineDisplacement'].apply(lambda x: float(x) if len(x) == 3 and x[1] == '.' else 'other')
train_old['engineDisplacement'].replace('other', 0.0, inplace=True)
train_old['engineDisplacement'].replace(0.0, train_old['engineDisplacement'].median(), inplace=True)

test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: str(x).replace('LTR', ''))
test['engineDisplacement'].replace(' ', 0.0, inplace=True)
test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: float(x))


# enginePower
train['enginePower'] = train['enginePower'].apply(lambda x: int(str(x[:-4])))
train_old['enginePower'] = train_old['enginePower'].apply(lambda x: int(x))
test['enginePower'] = test['enginePower'].apply(lambda x: int(str(x[:-4])))


# productionDate
train['productionDate'] = train['productionDate'].apply(lambda x: float(x))
train_old['productionDate'] = train_old['productionDate'].apply(lambda x: float(x))
test['productionDate'] = test['productionDate'].apply(lambda x: float(x))


# mileage
train['mileage'] = train['mileage'].apply(lambda x: int(x.replace(' ', '')[:-2]))

#train['mileage_epmty'] = train['mileage'].mask(train['mileage'] != 1, 0)
#train_old['mileage_epmty'] = train_old['mileage'].mask(train_old['mileage'] == 0, 1)
#train_old['mileage_epmty'] = train_old['mileage_epmty'].mask(train_old['mileage_epmty'] != 1, 0)
#test['mileage_epmty'] = test['mileage'].mask(test['mileage'] != 1, 0)

#train['mileage_epmty'] = train.mileage_epmty * ((2021-train.productionDate) * 20000 + 10000)
#train_old['mileage_epmty'] = train_old.mileage_epmty * ((2020-train_old.productionDate) * 20000 + 10000)
#test['mileage_epmty'] = test.mileage_epmty * ((2020-test.productionDate) * 20000 + 10000)

#train['mileage'] = train['mileage'] + train['mileage_epmty']
#train.drop(['mileage_epmty'], axis=1, inplace=True)
#train_old['mileage'] = train_old['mileage'] + train_old['mileage_epmty']
#train_old.drop(['mileage_epmty'], axis=1, inplace=True)
#test['mileage'] = test['mileage'] + test['mileage_epmty']
#test.drop(['mileage_epmty'], axis=1, inplace=True)


# modelDate
train['modelDate'] = train['modelDate'].apply(lambda x: int(float(x)))
train_old['modelDate'] = train_old['modelDate'].apply(lambda x: int(x))


# model_name
train['model_name'] = train['car_url'].apply(lambda x: x.split('/')[7])
train_old['model_name'] = train_old['model_name'].apply(lambda x: x.lower())
test['model_name'] = test['model_name'].apply(lambda x: x.lower())


# numberOfDoors
train['numberOfDoors'] = train['numberOfDoors'].apply(lambda x: int(float(x)))
train_old['numberOfDoors'] = train_old['numberOfDoors'].apply(lambda x: int(x))


# price
train['price'] = train['price'].apply(lambda x: int(float(x)))

# prices in train_old goes to prices in train
# usd_fixer = USD_TRAIN / USD_TRAIN_OLD
# train_old['price'] = train_old['price'].apply(lambda x: int(x * usd_fixer))

train['price_log'] = np.log(train['price'])
train_old['price_log'] = np.log(train_old['price'])

# vehicleTransmission
train_old['vehicleTransmission'] = train_old['vehicleTransmission'].map({
    'AUTOMATIC':'автоматическая',
    'MECHANICAL':'механическая',
    'ROBOT':'роботизированная',
    'VARIATOR':'вариатор'})


# vendor
vendors_dict = {
    'NISSAN': 'JP',
    'LEXUS': 'JP',
    'MERCEDES': 'DE',
    'BMW': 'DE',
    'VOLKSWAGEN': 'DE',
    'MITSUBISHI': 'JP',
    'TOYOTA': 'JP',
    'SKODA': 'EU',
    'HONDA': 'JP', 
    'AUDI': 'DE',
    'VOLVO': 'EU',
    'INFINITI': 'JP',
    'other': 'unknown'
}

train['vendor'] = train['brand'].map(vendors_dict)
train_old['vendor'] = train_old['brand'].map(vendors_dict)
test['vendor'] = test['brand'].map(vendors_dict)


# owners
owners_train = {
    '3 или более': 3.0,
    '2 владельца': 2.0,
    '1 владелец': 1.0
}

owners_test = {
    '3 или более': 3.0,
    '2\xa0владельца': 2.0,
    '1\xa0владелец': 1.0
}

train.replace({"owners": owners_train}, inplace=True)
test.replace({"owners": owners_test}, inplace=True)
train_old['owners'] = train_old['owners'].apply(lambda x: float(x))


# owning
train['owning'] = train['owning'].apply(lambda x: getOwningMonths(x))
test['owning'] = test['owning'].apply(lambda x: getOwningMonths(x))

df_temp = pd.DataFrame(train_old['owning'])
pattern = re.compile('\d{4}')
df_temp['year'] = df_temp['owning'].apply(lambda x:str(pattern.findall(str(x)))[2:-2])
pattern = re.compile('(\d{1,2})\}$')
df_temp['month'] = df_temp['owning'].apply(lambda x:str(pattern.findall(str(x)))[2:-2])

df_temp['year'] = df_temp['year'].map(lambda x: int(x) if len(x)==4 else 0)
df_temp['month'] = df_temp['month'].map(lambda x: int(x) if len(x)>0 else 0)
df_temp['delta_own'] = df_temp.apply(lambda x: calc_delta_ouned(x), axis=1)
train_old['owning'] = df_temp['delta_own']


# pts
train_old['pts'] = train_old['pts'].map({'ORIGINAL':'Оригинал', 'DUPLICATE':'Дубликат'})
train_old['pts'].fillna('Оригинал', inplace=True)


# wheel
train_old['wheel'] = train_old['wheel'].map({'RIGHT':'Правый', 'LEFT':'Левый'})


# remove unsuitable data
unused_cols = [
    'car_url',
    'complectation_dict',
    'equipment_dict',
    'image',
    'model_info',
    'parsing_unixtime',
    'sell_id',
    'super_gen',
]

train.drop(columns=unused_cols, axis=1, inplace=True)
test.drop(columns=unused_cols, axis=1, inplace=True)

### Контроль соответствия выборок и объединение тренировочных датасетов

In [ ]:
getFeaturesInfo(train)

In [ ]:
getFeaturesInfo(train_old)

In [ ]:
getFeaturesInfo(test)

In [ ]:
train = train.append(train_old, sort=False).reset_index(drop=True)
del train_old
gc.collect()

In [ ]:
train.sample(5)

# EDA

## Pandas Profiling and Basic Info

In [ ]:
# profile = train.profile_report()
# profile.to_file('data_report_train.html')

In [ ]:
# profile = test.profile_report()
# profile.to_file('data_report_test.html')

In [ ]:
train.describe()

In [ ]:
test.describe()

Следующие признаки либо дублируют информацию в прочих, либо содержат значительное количество пропусков, либо не подлежат приведению к единому формату между тестовой и тренировочной выборками:
* complectation_dict
* equipment_dict
* image
* name
* model_info
* super_gen
* vehicleConfiguration (комбинированный признак - тип кузова, количество дверей, тип коробки передач и объем двигателя)

### bodyType

Подавляющее большинство автомобилей - внедорожники и седаны. Семейные и коммерческие авто (компактвэны, фургоны, микровэны, универсалы) в целом дешевле, чем прочие. А вот родстеры, кабриолеты и, особенно, лимузины - дороже.

In [ ]:
train["bodyType"].value_counts().plot.barh()


In [ ]:
test["bodyType"].value_counts().plot.barh()


In [ ]:
get_boxplot(train, 'bodyType')

### brand

Тренировочный датасет содержит 36 марок. В тестовой выборке только 12: 'SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI', 'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'. Для повышения эффективности модели на тестовом датасете оставим 12 брендов, а оставшиеся объединим в категории "Другие".

Автомобили Мерседес, БМВ и Лексу, в целом, дорогие. Skoda, Honda в данной выборке оказываются бюджетными. В то же время категория "Другие" имеет еще более низкую среднюю цену - в том числе потому, что в ней есть и отечественные, и китайские автомобили.

In [ ]:
train["brand"].value_counts().plot.barh()

In [ ]:
test["brand"].value_counts().plot.barh()

In [ ]:
get_boxplot(train, 'brand')

### color

Преобразование признаков не требуется.

Наиболее часто встречающиеся автомобили - черные. Они же и более дорогие. Неожиданно примерно одинаковый разброс и среднюю цену имеют серые, оранжевые и коричневые авто.

In [ ]:
train["color"].value_counts().plot.barh()

In [ ]:
test["color"].value_counts().plot.barh()

In [ ]:
get_boxplot(train, 'color')

### description

Описание технического состояния ТС, выполненное продавцом. Без применения анализа текстов признак не применим.
Анализ и добавление признаков осуществим после объединения датасетов.

In [ ]:
# ... 

### engineDisplacement

Требовалось удаление символов 'LTR' в тестовой выборке. Ниже, при проведении корреляционного анализа, будет показано, что признак коррелирует с мощностью двигателя, что ожидаемо.

In [ ]:
train['engineDisplacement'].value_counts()

In [ ]:
test['engineDisplacement'].value_counts()

### enginePower

Наиболее часто встречаются мощности двигателя в 150 и 249 л.с. На основе enginePower будет построена наивная модель, которая будет предсказывать цену как медиану цен среди автомобилей с аналогичной мощностью двигателя.

In [ ]:
train['enginePower'].value_counts()

In [ ]:
test['enginePower'].value_counts()

### fuelType - преобразование не требуется

Подавляющее большинство автомобилей - бензиновые. Доля гибридов, электрических и на газу незначительна. Электрические автомобиле выставляются по ценам выше рынка, в то время как газовые - ниже (однако, выборки по этим категориям небольшие, и вероятно, что нерепрезентативны).

In [ ]:
get_boxplot(train, 'fuelType')

In [ ]:
train["fuelType"].value_counts().plot.barh()

In [ ]:
test["fuelType"].value_counts().plot.barh()

### productionDate

В выборках есть один автомобиль 1904 года. В то же время, наибольшее количество автомобилей - автомобили возрастом 8-10 лет.

In [ ]:
pd.DataFrame([train.productionDate.describe(), test.productionDate.describe()])

### mileage

TODO: Deal with 1km-mileage

In [ ]:
display(len(train[train.mileage==1]))
display(len(test[test.mileage==1]))

In [ ]:
train['mileage'].sort_values().hist()

In [ ]:
test['mileage'].sort_values().hist()

### modelDate

Дата производства и дата выпуска модели в свет - признаки коррелирующиеся, но на их основе в дальнейшем введем признак "Время, в течение которого модель находится на рынке" (жизненный цикл модели).

In [ ]:
train['modelDate'].sort_values().hist(bins=30)

In [ ]:
test['modelDate'].sort_values().hist(bins=30)

In [ ]:
# модели до 1980 года
display(len(train[train.modelDate < 1980]))
display(len(test[test.modelDate < 1980]))

### model_name

Признак будет использован при создании новых признаков - редкость модели, ассортимент автомобильного бренда.

In [ ]:
train['model_name'].value_counts()

In [ ]:
test['model_name'].value_counts()

### numberOfDoors

In [ ]:
train['numberOfDoors'].value_counts()

In [ ]:
train[train['numberOfDoors'] == 0].T

In [ ]:
test['numberOfDoors'].value_counts()

In [ ]:
test[test['numberOfDoors'] == 0].T

### price

Значения целевой переменной имеют высокий разброс. Для улучшения модели при работе с такими данными будут полезны логарифмирование, работа с выбросами и стандартизация.

In [ ]:
train.price.plot()

In [ ]:
train.price_log.plot()

In [ ]:
train['price'].sort_values()

### vehicleTransmission

Автомобилей на механической коробке - четверть тренировочного датасета. И они, в целом, значительно дешевле прочих. В то время как средний уровень цен и размах для автомобилей с автоматическими, роботизированными КПП и вариаторами практически идентичный.

In [ ]:
get_boxplot(train, 'vehicleTransmission')

In [ ]:
train['vehicleTransmission'].value_counts()

In [ ]:
test['vehicleTransmission'].value_counts()

### vendor

Подтверждается то, что мы видели при анализе распределения цен по брендам производителей. немецкий автомобили - самые дорогие. Европейские (за исключением немецких) - дешевле японских.

In [ ]:
get_boxplot(train, 'vendor')

In [ ]:
train['vendor'].value_counts()

In [ ]:
test['vendor'].value_counts()

### owners

In [ ]:
# Учитывая зависимость пробега и количество владельцев,
# посчитал среднее значение владельцев в зависимости от пробега.
# Отсюда можно выделить 3 переломных цифры,когда в среднем владелец меняется.
bins = np.arange(0, 170000, 10000)
train.groupby(pd.cut(train.mileage, bins)).owners.median()

In [ ]:
train['owners'].value_counts()

In [ ]:
test['owners'].value_counts()

### owning

In [ ]:
train['owning'].sort_values().hist(bins=20)

In [ ]:
test['owning'].sort_values().hist(bins=20)

In [ ]:
# more than 10 years with one owner
display(len(train[train.owning > 120]))
display(len(test[test.owning > 120]))

### pts

Отсутствие оригинала ПТС серьезно влияет на цену в объявлении. Отличный бинарный признак.

In [ ]:
pts_duplicates = (len(train[train.pts == 'Дубликат']) + len(test[test.pts == 'Дубликат'])) / (len(train) + len(test))
print('Процент автомобилей с дубликатом ПТС:', round(100*pts_duplicates, 2))

In [ ]:
get_boxplot(train, 'pts')

### drive

При учете того, что общая доля количества полноприводных и переднеприводных автомобилей отличаются незначительно (и это наиболее популярные категории), разброс цен между ними существенный и в пользу полноприводных автомобилей.

In [ ]:
get_boxplot(train, 'drive')

In [ ]:
train["drive"].value_counts().plot.barh()

In [ ]:
test["drive"].value_counts().plot.barh()

### wheel

Праворульных автомобилей мало и, как правило, они дешевле.

In [ ]:
get_boxplot(train, 'wheel')

In [ ]:
train["wheel"].value_counts().plot.barh()

In [ ]:
test["wheel"].value_counts().plot.barh()

# FEATURE ENGINEERING

При получении данных об объявлениях с сайта можно дополнительно получать информацию, например, о регионе, в котором размещено объявление, а также количестве просмотров объявления. Тем не менее, в тестовой выборке эти сведения отсутствуют, потому парсинг по этим позициям не осуществлялся.

### brand model range

Количество моделей в линейке автопроизводителя

In [ ]:
brand_range_dict = dict(train.groupby('brand')['model_name'].nunique())
train['brand_range'] = train['brand'].map(brand_range_dict)
brand_range_dict = dict(test.groupby('brand')['model_name'].nunique())
test['brand_range'] = test['brand'].map(brand_range_dict)


In [ ]:
sns.jointplot(x='brand_range', y='price_log', data=train, kind='reg')

### brand rarity

Редкость (или популярность) конкретной марки автомобиля.

In [ ]:
brand_rar_dict = dict(train['brand'].value_counts() / len(train))
train['brand_pop'] = train['brand'].map(brand_rar_dict)
brand_rar_dict = dict(test['brand'].value_counts() / len(test))
test['brand_pop'] = test['brand'].map(brand_rar_dict)


In [ ]:
sns.jointplot(x='brand_pop', y='price_log', data=train, kind='reg')

### model rarity

Редкость (или популярность) конкретной модели автомобиля из объявления.

In [ ]:
model_rar_dict = dict(train['model_name'].value_counts() / len(train))
train['model_pop'] = train['model_name'].map(model_rar_dict)
model_rar_dict = dict(test['model_name'].value_counts() / len(test))
test['model_pop'] = test['model_name'].map(model_rar_dict)


In [ ]:
sns.jointplot(x='model_pop', y='price_log', data=train, kind='reg')

### model_in_brand

Признак комбинирует в себе широу ассортимента автопроизводителя и популярность конкретной модели.

Не прошел проверку на мультиколлинеарность

In [ ]:
#train['model_cars_count'] = train['model_name'].apply(lambda x: train['model_name'].value_counts()[x])
#train['brand_cars_total'] = train['brand'].apply( lambda x: train.groupby('brand')['model_name'].value_counts()[x].sum())
#train['model_in_brand'] = train['model_cars_count'] / train['brand_cars_total']
#train.drop(['model_cars_count', 'brand_cars_total'], axis=1, inplace=True)

#test['model_cars_count'] = test['model_name'].apply(lambda x: test['model_name'].value_counts()[x])
#test['brand_cars_total'] = test['brand'].apply( lambda x: test.groupby('brand')['model_name'].value_counts()[x].sum())
#test['model_in_brand'] = test['model_cars_count'] / test['brand_cars_total']
#test.drop(['model_cars_count', 'brand_cars_total'], axis=1, inplace=True)


### model_age

Возраст модели (не конкретного автомобиля).

Не прошел проверку на мультиколлинеарность

In [ ]:
#train['model_age'] = 2021 - train['modelDate']
#test['model_age'] = 2021 - test['modelDate']


### car_age

Преобразование даты производства в возраст автомобиля.

In [ ]:
train['car_age'] = 2021 - train['productionDate']
test['car_age'] = 2021 - test['productionDate']


# Разобьем возраст на интервалы
# bins = [0, 3, 6, 13, 20, 35, 120]
# numbers = ['3', '6','13','20','35', '116']

# train['age_intervals'] = pd.cut(train['car_age'],bins, labels=numbers)
# test['age_intervals'] = pd.cut(test['car_age'], bins, labels=numbers)

### prod_epoch

Период, в течение которого производилась конкретная модель.

In [ ]:
train['prod_epoch'] = train['productionDate'] - train['modelDate']
test['prod_epoch'] = test['productionDate'] - test['modelDate']

In [ ]:
sns.jointplot(x='prod_epoch', y='price_log', data=train, kind='reg')

### seller type

Определение типа продавца по ключевым словам из описания. Два флага - собственник и автосалон.

In [ ]:
train['description'] = train['description'].map(lambda x: str(x).lower())

train['owner'] = train['description'].apply(lambda x: 1 if 'торг' in x or 'не нуждаюсь' in x or len(x)<200 else 0)
train['showroom'] = train['description'].apply(lambda x: 1 if 'traid-in' in x or 'дилер' in x or 'в кредит' in x or 'клиент' in x
                                               or 'без комис' in x or 'страховани' in x or 'трейд-ин' in x 
                                               or 'в наличии' in x or 'выгодное пр' in x or 'автокредит' in x else 0)
display(train['owner'].value_counts())
display(train['showroom'].value_counts())

In [ ]:
test['description'] = test['description'].map(lambda x: str(x).lower())

test['owner'] = test['description'].apply(lambda x: 1 if 'торг' in x or 'не нуждаюсь' in x or len(x)<200 else 0)
test['showroom'] = test['description'].apply(lambda x: 1 if 'traid-in' in x or 'дилер' in x or 'в кредит' in x or 'клиент' in x
                                             or 'без комис' in x or 'страховани' in x or 'трейд-ин' in x 
                                             or 'в наличии' in x or 'выгодное пр' in x or 'автокредит' in x else 0)
display(test['owner'].value_counts())
display(test['showroom'].value_counts())

### Collections, Sport

Создадим несколько бинарных признаков исходя из модели, которые на практике сильно влияют на стоимость автомобиля:

* collector - коллекционные автомобили
* sport - спортивные автомобили

In [ ]:
# список коллекционных марок (из машин произведеных до 1960 года )
collector = list(train.query('productionDate<1980').model_name.value_counts().keys())
# список спортивных марок(из моделей которые в среднем имеют мощность более 500 л.с.)
sport = list(train.groupby('model_name').enginePower.mean().sort_values(ascending=False)[:25].keys())
# закодируем
for value, models in {'collector': collector, 'sport': sport}.items():
    mask = train['model_name'].isin(models)
    train[value] = mask*1
    
    
# список коллекционных марок (из машин произведеных до 1960 года )
collector = list(test.query('productionDate<1980').model_name.value_counts().keys())
# список спортивных марок(из моделей которые в среднем имеют мощность более 500 л.с.)
sport = list(test.groupby('model_name').enginePower.mean().sort_values(ascending=False)[:25].keys())
# закодируем
for value, models in {'collector': collector, 'sport': sport}.items():
    mask = test['model_name'].isin(models)
    test[value] = mask*1

### description analysis

TODO: Word2Vec

Создадим из описания(description) несколько числовых признаков:

* количество символов из описания (description_length)
* повторяемость объявлений (repeat_description)

In [ ]:
train['description_length'] = train['description'].map(lambda x: len(x))

dict_count_description = train['description'].value_counts().loc[lambda x: x > 2].to_dict()
train['repeat_description'] = train['description'].map(dict_count_description)
train['repeat_description'].fillna(1, inplace=True)


test['description_length'] = test['description'].map(lambda x: len(x))

dict_count_description = test['description'].value_counts().loc[lambda x: x > 2].to_dict()
test['repeat_description'] = test['description'].map(dict_count_description)
test['repeat_description'].fillna(1, inplace=True)

Проведем анализа текста объявлений на предмет частоты встречаемости конкретных словосочетаний и сформируем несколько признаков на основе ключевых слов.

In [ ]:
text = ' '.join(train['description'].values)
text_trigrams = [i for i in ngrams(text.split(' '), 5)]
text_trigrams
my_list = Counter(text_trigrams).most_common(30)
my_list

In [ ]:
comment_words = '' 
stopwords = set(STOPWORDS) 
newStopWords = ['автомобиль', 'продаю', 'автомобилей', 'пробег']
stopwords.update(newStopWords)

  
# iterate through the csv file 
for val in train.description: 
    # typecaste each val to string 
    val = str(val) 
    # split the value 
    tokens = val.split() 
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower()
        
    comment_words += " ".join(tokens)+" "
    

wordcloud = WordCloud(
    background_color='white',
    stopwords=stopwords,
    max_words=300,
    max_font_size=200, 
    width=1000, height=800,
    random_state=42,
).generate(" ".join(train['description'].astype(str)))

# plot the WordCloud image                        
plt.figure(figsize = (12, 14), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show()

In [ ]:
text = ' '.join(test['description'].values)
text_trigrams = [i for i in ngrams(text.split(' '), 5)]
text_trigrams
my_list = Counter(text_trigrams).most_common(30)
my_list

In [ ]:
comment_words = '' 
stopwords = set(STOPWORDS) 
newStopWords = ['автомобиль', 'продаю', 'автомобилей', 'пробег']
stopwords.update(newStopWords)

  
# iterate through the csv file 
for val in test.description: 
    # typecaste each val to string 
    val = str(val) 
    # split the value 
    tokens = val.split() 
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
    
    
wordcloud = WordCloud(
    background_color='white',
    stopwords=stopwords,
    max_words=300,
    max_font_size=200, 
    width=1000, height=800,
    random_state=42,
).generate(" ".join(test['description'].astype(str)))

# plot the WordCloud image                        
plt.figure(figsize = (12, 14), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show()

In [ ]:
# under comments - multi-high-corr-features

# train['is_pillows'] = train.description.apply(lambda x: 1 if 'подушки безопасности' in x else 0)
# test['is_pillows'] = test.description.apply(lambda x: 1 if 'подушки безопасности' in x else 0)

# train['is_wheelsup'] = train.description.apply(lambda x: 1 if 'усилитель руля' in x else 0)
# test['is_wheelsup'] = test.description.apply(lambda x: 1 if 'усилитель руля' in x else 0)

train['is_great'] = train.description.apply(lambda x: 1 if 'отличном состоянии' in x else 0)
test['is_great'] = test.description.apply(lambda x: 1 if 'отличном состоянии' in x else 0)

train['is_good'] = train.description.apply(lambda x: 1 if 'хорошем состоянии' in x else 0)
test['is_good'] = test.description.apply(lambda x: 1 if 'хорошем состоянии' in x else 0)

# train['is_ckey'] = train.description.apply(lambda x: 1 if 'центральный замок' in x else 0)
# test['is_ckey'] = test.description.apply(lambda x: 1 if 'центральный замок' in x else 0)

# train['is_elecwindow'] = train.description.apply(lambda x: 1 if 'электростеклоподъемники' in x else 0)
# test['is_elecwindow'] = test.description.apply(lambda x: 1 if 'электростеклоподъемники' in x else 0)

# train['is_pc'] = train.description.apply(lambda x: 1 if 'бортовой компьютер' in x else 0)
# test['is_pc'] = test.description.apply(lambda x: 1 if 'бортовой компьютер' in x else 0)

train['is_climate'] = train.description.apply(lambda x: 1 if 'климат контроль' in x else 0)
test['is_climate'] = test.description.apply(lambda x: 1 if 'климат контроль' in x else 0)

train['is_cruise'] = train.description.apply(lambda x: 1 if 'круиз контроль' in x else 0)
test['is_cruise'] = test.description.apply(lambda x: 1 if 'круиз контроль' in x else 0)

train['is_antiblock'] = train.description.apply(lambda x: 1 if 'антиблокировочная система' in x else 0)
test['is_antiblock'] = test.description.apply(lambda x: 1 if 'антиблокировочная система' in x else 0)


In [ ]:
equip = [ 'is_great', 'is_good',
          'is_climate', 'is_cruise', 'is_antiblock']

for eq in equip:
    plt.figure(figsize=(8, 5))
    g1 = sns.boxplot(y='price_log', x=eq, 
                      data=train, color='red')
    g1.set_title(f"price_log for {eq}", fontsize=20)

    g1.set_ylabel("price_log", fontsize=15)
    g1.set_xticklabels(g1.get_xticklabels(),rotation=45)
    plt.show()

### tax_rate

Попытка добавить признак, характеризующий налоговое бремя, которое будет возложено на автовладельца после приобретения машины.

In [ ]:
# unsuitable due to high multi-corr with enginePower
# show bigger MAPE against using enginePower-feature

# Разобьем мощность на интервалы
# bins = [10, 35, 100, 125, 150, 175, 200, 225, 250, 801]
# numbers = ['5000','12','25','35', '45', '50', '65', '75', '150']

# налоговая база
# train['tax_base'] = pd.cut(train['enginePower'], bins, labels=numbers)
# test['tax_base'] = pd.cut(test['enginePower'], bins, labels=numbers)
# train['tax_base'] = train['tax_base'].values.astype('int64')
# test['tax_base'] = test['tax_base'].values.astype('int64')

# train['tax'] = train['enginePower'] * train['tax_base']
# test['tax'] = test['enginePower'] * test['tax_base']

Попытка неудачная - признак сильно коррелирует с мощностью двигателя, при этом статистическая значимость мощности двигателя при проверке оказалась выше.

In [ ]:
# Посмотрим наибольшие корреляционные связи с  переменной price
all_cols = set(train.columns) - set(['price', 'price_log'])
correlations = train[all_cols].corrwith(train['price']).sort_values(ascending=False)
plot = sns.barplot(y=correlations.index, x=correlations)

# LABEL ENCODING

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест

data = test.append(train, sort=False).reset_index(drop=True) # объединяем

# drop surplus features (come later to analyze them) 
surplus_cols = [
    'description',
    'model_name',
    'name',
    'vehicleConfiguration',
    'modelDate',
    'productionDate',
    'price_log'
]

data.drop(surplus_cols, axis=1, inplace=True)

y = train['price'].values

del [[train, test]]
gc.collect()

cat_cols = [
    'bodyType',
    'brand',
    'color',
    'fuelType',
    'vehicleTransmission',
    'vendor',
    'owners',
    'pts',
    'drive',
    'wheel'
]

for colum in cat_cols:
    data[colum] = data[colum].astype('category').cat.codes
    
getFeaturesInfo(data)

### Feature Importance

TODO: Fix memory leaks in Binary Features Importance Script

In [ ]:
data_temp = data.loc[data['sample'] == 1].drop(['sample'], axis=1)

In [ ]:
num_cols = [
    'engineDisplacement',
    'enginePower',
    'mileage',
    'numberOfDoors',
    'owning',
    'brand_range',
    'brand_pop',
    'model_pop',
    'prod_epoch',
    'car_age',
    'description_length',
    'repeat_description'
]

bin_cols = [
    'pts',
    'wheel',
    'owning_empty',
    'owner',
    'showroom',
    'collector', 'sport',
    'is_great', 'is_good',
    'is_climate', 'is_cruise', 'is_antiblock'
]

cat_cols = [
    'bodyType',
    'brand',
    'color',
    'owners',
    'fuelType',
    'vehicleTransmission',
    'vendor',
    'drive'
]

In [ ]:
imp_num = pd.Series(f_classif(data_temp[num_cols], data_temp['price'])[0], index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind = 'barh', color='pink', title='Numeric Features Importance')

In [ ]:
imp_num

In [ ]:
#for part in [bin_cols_1, bin_cols_2, bin_cols_3]:
#    imp_bin = pd.Series(mutual_info_classif(data_temp[part], data_temp['price'], discrete_features=False), index=part)
#    imp_bin.sort_values(inplace=True)
#    imp_bin.plot(kind='barh', color='pink', title='Binary Features Importance')
#    display(imp_bin)
#    del [imp_bin]
#    gc.collect()

In [ ]:
imp_cat = pd.Series(mutual_info_classif(
    data_temp[cat_cols],
    data_temp['price'],
    discrete_features=True
), index=cat_cols)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh', color='pink', title='Categorical Features Importance')

In [ ]:
imp_cat

### Выбросы и стандартизация

In [ ]:
for col in num_cols:
    median = data[col].median()
    IQR = data[col].quantile(0.75) - data[col].quantile(0.25)
    perc25 = data[col].quantile(0.25)
    perc75 = data[col].quantile(0.75)
    
    print("Column: ", col)
    print(' 25%: {:.4},\n'.format(perc25), '75%: {:.4},\n'.format(perc75),
          "IQR: {:.4}, \n".format(IQR), "Borderline: [{f:.4}, {l:.4}].\n".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))
    print()
    
    # replace outliers with border-values
    data[col] = np.where(data[col] > (perc75 + 1.5*IQR), (perc75 + 1.5*IQR), data[col])
    data[col] = np.where(data[col] < (perc25 - 1.5*IQR), (perc25 - 1.5*IQR), data[col])

In [ ]:
# scaler = StandardScaler() less effective
scaler = MinMaxScaler()
data[num_cols] = scaler.fit_transform(data[num_cols].values)

### Get Dumies

In [ ]:
data = pd.get_dummies(data, prefix=cat_cols, columns=cat_cols)

### Correlation Matrix

In [ ]:
plt.title('Correlation Matrix of dataset features')
plt.rcParams['figure.figsize'] = (30,20)
sns.heatmap(data.corr(), vmin=-1, vmax=1, annot = False)

In [ ]:
# show features with corr > 0.7
corr = data.corr().abs()
corr_table = corr.unstack()
corr_table_sorted = corr_table.sort_values(kind="quicksort", ascending=False)
corr_exclude_ones = corr_table_sorted[corr_table_sorted != 1]
high_corr = corr_exclude_ones[corr_exclude_ones >= 0.7]
high_corr

In [ ]:
# exclude features due high corr and low importance
data.drop([
    'vendor_3', 'brand_12', 'fuelType_3',
    'brand_pop', 'drive_2', 'brand_8', 'owning_empty', 'repeat_description'
], axis=1, inplace=True)

### Query

In [ ]:
X = data.query('sample == 1').drop(['sample', 'price'], axis=1)
X_sub = data.query('sample == 0').drop(['sample', 'price'], axis=1)

## Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Models

### Naive Model

Создадим "наивную" модель. Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [ ]:
# tmp_train = X_train.copy()
# tmp_train['price'] = y_train

# Находим median по экземплярам enginePower в трейне и размечаем тест
# predict = X_test['enginePower'].map(tmp_train.groupby('enginePower')['price'].median())

# оцениваем точность
# print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

# MAPE: 62.67%

### CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

In [ ]:
# model = CatBoostRegressor(iterations = 5000,
#                           random_seed = RANDOM_SEED,
#                           eval_metric='MAPE',
#                           custom_metric=['R2', 'MAE'],
#                           silent=True,
#                           )
# model.fit(X_train, y_train,
#          eval_set=(X_test, y_test),
#          verbose_eval=0,
#          use_best_model=True)

# model.save_model('catboost_single_model_baseline.model')
# 
# оцениваем точность
# predict = model.predict(X_test)
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

# MAPE: 16.04%

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [ ]:
# np.log(y_train)

In [ ]:
# model = CatBoostRegressor(iterations = 5000,
#                           random_seed = RANDOM_SEED,
#                           eval_metric='MAPE',
#                           custom_metric=['R2', 'MAE'],
#                           silent=True,
#                           )
# model.fit(X_train, np.log(y_train),
#          eval_set=(X_test, np.log(y_test)),
#          verbose_eval=0,
#          use_best_model=True)

# model.save_model('catboost_single_model_2_baseline.model')

# predict_test = np.exp(model.predict(X_test))
# predict_submission = np.exp(model.predict(X_sub))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# MAPE: 12.89%

## ПОДБОР ПАРАМЕТРОВ

### CatBoost Regressor

In [ ]:
#start = datetime.now()
#
#model = CatBoostRegressor(iterations=50,
#                          loss_function='MAPE',
#                          metric_period=10)
#
#grid = {'learning_rate': [ 0.13, 0.14, 0.15],
#        'depth': [12],
#        'l2_leaf_reg': [7, 7.5, 8],
#        'random_strength': [0.3]}
#
#grid_search_result = model.grid_search(grid,
#                                       X=X_train,
#                                       y=np.log(y_train),
#                                       verbose=False)
#
#print('\nВремя выполнения - ', datetime.now() - start)
#print('\nНаилучшие параметры: ', grid_search_result['params'])

In [ ]:
# cb = CatBoostRegressor(iterations = 5000,
#                        random_seed = RANDOM_SEED,
#                        eval_metric='MAPE',
#                        custom_metric=['R2', 'MAE'],
#                        silent=True,
#                        learning_rate=0.15, depth=12,
#                        l2_leaf_reg=7.5, random_strength=0.3)

# cb.fit(X_train, np.log(y_train),
#        eval_set=(X_test, np.log(y_test)),
#        verbose=False,
#        use_best_model=True)

# cb.save_model('catboost_log_model.model')

# оцениваем точность
# predict_cb = np.exp(cb.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_cb))*100:0.2f}%")

# MAPE: 12.88%

### LGBM Regressor

In [ ]:
lgbmr = LGBMRegressor(objective='regression', max_depth=12, num_leaves=1000,
                      learning_rate=0.02, n_estimators=22000, metric='mape',
                      feature_fraction=0.6)

lgbmr.fit(X_train, np.log(y_train))

# оцениваем точность
predict_lgbmr = np.exp(lgbmr.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_lgbmr))*100:0.2f}%")

# MAPE: 12.89%

In [ ]:
# TODO

# from sklearn.model_selection import GridSearchCV

# lgbmr = LGBMRegressor(objective='regression', metric='mape', feature_fraction=0.6, random_state=RANDOM_SEED)

# params = {
#     'num_leaves': [10, 25, 50, 100, 250],
#     'learning_rate': [0.01, 0.1, 1.0],
#     'max_depth': [-1, 3, 7, 9, 13],
#     'n_estimators': [50, 100, 500, 1000],
#     'boosting_type': ['gbdt', 'dart', 'goss']
# }

# grid = GridSearchCV(lgbmr, params, scoring='neg_mean_absolute_error', cv=5)
# grid.fit(X_train, y_train)

# print(grid.best_estimator_)
# print(grid.best_params_)

In [ ]:
# start = datetime.now()

# bagg_lgbm = BaggingRegressor(lgbmr, n_estimators=3, n_jobs=4, random_state=RANDOM_SEED)
# bagg_lgbm.fit(X_train, np.log(y_train))
# predict_bagg_lgbm = np.exp(bagg_lgbm.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_bagg_lgbm))*100:0.2f}%")

### Random Forest

In [ ]:
#start = datetime.now()
#
#random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 400, num = 4)],
#               'max_features': ['auto', 'sqrt'],
#               'max_depth': [int(x) for x in np.linspace(5, 15, num = 6)] + [None],
#               'min_samples_split': [2, 5, 10],
#               'min_samples_leaf': [1, 2, 4],
#               'bootstrap': [True, False]}
#
#rfr = RandomForestRegressor(random_state = RANDOM_SEED)
#rf_random = RandomizedSearchCV(estimator = rfr,
#                               param_distributions = random_grid,
#                               n_iter = 100,
#                               cv = 3,
#                               verbose=10,
#                               random_state=RANDOM_SEED,
#                               n_jobs=4)
#
#rf_random.fit(X_train, np.log(y_train))
#print(rf_random.best_params_)
#print('\nВремя выполнения - ', datetime.now() - start)


In [ ]:
# best_rfr = RandomForestRegressor(random_state=RANDOM_SEED,
#                                  n_estimators=200,
#                                  min_samples_split=2,
#                                  min_samples_leaf=2,
#                                  max_features='auto',
#                                  max_depth=None,
#                                  bootstrap=True)

# best_rfr.fit(X_train, np.log(y_train))

# оцениваем точность
# predict_rfr = np.exp(best_rfr.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_rfr))*100:0.2f}%")

# MAPE: 13.39%

### Gradient Boosting Regressor

In [ ]:
#start = datetime.now()
#
#random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 400, num = 8)],
#               'max_features': ['auto', 'sqrt', 'log2'],
#               'max_depth': [int(x) for x in np.linspace(5, 15, num = 6)] + [None],
#               'min_samples_split': [2, 5, 10],
#               'min_samples_leaf': [1, 2, 4]}

#gbr = GradientBoostingRegressor()
#gbr_random = RandomizedSearchCV(estimator = gbr,
#                                param_distributions = random_grid,
#                                n_iter = 100,
#                                cv = 3,
#                                verbose=10,
#                                random_state=RANDOM_SEED,
#                                n_jobs=4)

#gbr_random.fit(X_train, np.log(y_train))
#print(gbr_random.best_params_)
#print('\nВремя выполнения - ', datetime.now() - start)


In [ ]:
# best_gbr = GradientBoostingRegressor(random_state=RANDOM_SEED,
#                                      n_estimators=400,
#                                      min_samples_split=5,
#                                      min_samples_leaf=2,
#                                      max_features='auto',
#                                      max_depth=9)

# best_gbr.fit(X_train, np.log(y_train))

# оцениваем точность
# predict_gbr = np.exp(best_gbr.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_gbr))*100:0.2f}%")

# MAPE: 12.89%

### XGB Regressor

In [ ]:
# start = datetime.now()

# xgb_reg = xgb.XGBRegressor(objective='reg:squarederror',
#                            colsample_bytree=0.5,
#                            learning_rate=0.05,
#                            max_depth=12,
#                            alpha=1,
#                            n_estimators=1000)

# xgb_reg.fit(X_train, np.log(y_train))

# оцениваем точность
# predict_xgb = np.exp(xgb_reg.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_xgb))*100:0.2f}%")
# print('\nВремя выполнения - ', datetime.now() - start)

# MAPE: 12.54%

## BAGGING

### Random Forest

In [ ]:
# start = datetime.now()

# bagg_rfr = BaggingRegressor(best_rfr, n_estimators=3, n_jobs=4, random_state=RANDOM_SEED)
# bagg_rfr.fit(X_train, np.log(y_train))
# predict_bagg_rfr = np.exp(bagg_rfr.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_bagg_rfr))*100:0.2f}%")
# print('\nВремя выполнения - ', datetime.now() - start)

# MAPE: 13.62%

### Gradient Boosting

In [ ]:
# start = datetime.now()

# bagg_gbr = BaggingRegressor(best_gbr, n_estimators=3, n_jobs=4, random_state=RANDOM_SEED)
# bagg_gbr.fit(X_train, np.log(y_train))
# predict_bagg_gbr = np.exp(bagg_gbr.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_bagg_gbr))*100:0.2f}%")

# MAPE: 12.89%

### XGBoost

In [ ]:
# start = datetime.now()

# bagg_xgb = BaggingRegressor(xgb_reg, n_estimators=3, n_jobs=4, random_state=RANDOM_SEED)
# bagg_xgb.fit(X_train, np.log(y_train))
# predict_bagg_xgb = np.exp(bagg_xgb.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_bagg_xgb))*100:0.2f}%")
# print('\nВремя выполнения - ', datetime.now() - start)

# MAPE: 12.69%

## Stacking & Ensembles

In [ ]:
# start = datetime.now()

# estimators = [('b_gbr', BaggingRegressor(GradientBoostingRegressor(random_state=RANDOM_SEED,
#                                                                    n_estimators=400,
#                                                                    min_samples_split=5,
#                                                                    min_samples_leaf=2,
#                                                                    max_features='auto',
#                                                                    max_depth=9),
#                                          n_estimators=3,
#                                          n_jobs=4,
#                                          random_state=RANDOM_SEED)),
#               ('xgb', xgb.XGBRegressor(objective='reg:squarederror',
#                                        colsample_bytree=0.5,
#                                        learning_rate=0.05,
#                                        max_depth=12,
#                                        alpha=1,
#                                        n_estimators=1000))]

# st_ensemble = StackingRegressor(estimators=estimators,
#                                 final_estimator = CatBoostRegressor(iterations = 5000,
#                                                                     random_seed = RANDOM_SEED,
#                                                                     eval_metric='MAPE',
#                                                                     custom_metric=['R2', 'MAE'],
#                                                                     silent=True,
#                                                                     learning_rate=0.13,
#                                                                     depth=12,
#                                                                     l2_leaf_reg=7.5,
#                                                                     random_strength=0.3))

    
# st_ensemble.fit(X_train, np.log(y_train))

# predict_ensemble = np.exp(st_ensemble.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_ensemble))*100:0.2f}%")
# print('\nВремя выполнения - ', datetime.now() - start)

# MAPE: 12.93%

Осуществлен перебор регрессионных моделей из разных библиотек. Сделан подбор параметров. Попробован бэггинг и стэкинг, ансамбли моделей. Результаты экспериментов записаны в виде метрик под лучшими вариантами моделей и испробованы на тестовом датасете на каггле (перфоманс моделей на тестовом датасете - ниже).

### Lazy Predict

In [ ]:
# !pip install lazypredict 

In [ ]:
# from lazypredict.Supervised import LazyRegressor

# reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
# models, predictions = reg.fit(X_train, X_test, y_train, y_test)

# print(models)

# SUBMISSION

In [ ]:
# sample_submission['price'] = predict_submission
# sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
# Kaggle MAPE 19.19103

# train-dollar prices goes to test-dollar prices
# usd_fixer = USD_TRAIN / USD_TEST
# sample_submission['price'] = predict_submission * usd_fixer
# sample_submission.to_csv(f'submission_2_v{VERSION}_currency_correct.csv', index=False)
# Kaggle MAPE 17.26228

# round to nearest 5000 rub price w/ dollar-changing pattern
# usd_fixer = USD_TRAIN / USD_TEST
# sample_submission['price'] = np.round(predict_submission * usd_fixer,-3).astype('int')
# sample_submission.to_csv(f'submission_2_v{VERSION}_currency_correct_round.csv', index=False)
# Kaggle MAPE 17.26256

# catboost
# predict_test = np.exp(cb.predict(X_sub))
# sample_submission['price'] = predict_test
# sample_submission.to_csv(f'submission_CB_v{VERSION}.csv', index=False)
# Kaggle MAPE 18.00332

# LGBM Regressor
# predict_lgbmr = np.exp(lgbmr.predict(X_sub))
# sample_submission['price'] = predict_lgbmr
# sample_submission.to_csv(f'submission_LGBMR_v{VERSION}.csv', index=False)
# Kaggle MAPE 16.83216

# LGBM Regressor with currency fixer
usd_fixer = USD_TRAIN / USD_TEST
predict_lgbmr = np.exp(lgbmr.predict(X_sub))
sample_submission['price'] = predict_lgbmr * usd_fixer
sample_submission.to_csv(f'submission_LGBMR_v{VERSION}_currency_correct.csv', index=False)
# Kaggle MAPE 15.18838

# random forest
# predict_rfr = np.exp(best_rfr.predict(X_sub))
# sample_submission['price'] = predict_rfr
# sample_submission.to_csv(f'submission_RFR_v{VERSION}.csv', index=False)
# Kaggle MAPE 17.41994

# grad boosting
# predict_gbr = np.exp(best_gbr.predict(X_sub))
# sample_submission['price'] = predict_gbr
# sample_submission.to_csv(f'submission_GBR_v{VERSION}.csv', index=False)
# Kaggle MAPE 18.33223

# xgboost
# xgb_reg_pred = np.exp(xgb_reg.predict(X_sub))
# sample_submission['price'] = xgb_reg_pred
# sample_submission.to_csv(f'submission_XGB_v{VERSION}.csv', index=False)
# Kaggle MAPE 17.28800

# xgboost with currency fixer
# usd_fixer = USD_TRAIN / USD_TEST
# xgb_reg_pred = np.exp(xgb_reg.predict(X_sub))
# sample_submission['price'] = xgb_reg_pred * usd_fixer
# sample_submission.to_csv(f'submission_XGB_v{VERSION}_currency_correct.csv', index=False)
# Kaggle MAPE 15.83164

# bagging random forest
# predict_bagg_rfr = np.exp(bagg_rfr.predict(X_sub))
# sample_submission['price'] = predict_bagg_rfr
# sample_submission.to_csv(f'submission_bagRF_v{VERSION}.csv', index=False)
# Kaggle MAPE 17.09522

# bagging random forest with currency fixer
# usd_fixer = USD_TRAIN / USD_TEST
# predict_bagg_rfr = np.exp(bagg_rfr.predict(X_sub))
# sample_submission['price'] = predict_bagg_rfr * usd_fixer
# sample_submission.to_csv(f'submission_bagRF_v{VERSION}_currency_correct.csv', index=False)
# Kaggle MAPE

# bagging grad boosting
# predict_bagg_gbr = np.exp(bagg_gbr.predict(X_sub))
# sample_submission['price'] = predict_bagg_gbr
# sample_submission.to_csv(f'submission_bagGBR_v{VERSION}.csv', index=False)
# Kaggle MAPE 17.86295

# bagging xgboost
# predict_bagg_xgb = np.exp(bagg_xgb.predict(X_sub))
# sample_submission['price'] = predict_bagg_xgb
# sample_submission.to_csv(f'submission_bagXGB_v{VERSION}.csv', index=False)
# Kaggle MAPE 17.10128

# bagging xgboost with currency fixer
# usd_fixer = USD_TRAIN / USD_TEST
# predict_bagg_xgb = np.exp(bagg_xgb.predict(X_sub))
# sample_submission['price'] = predict_bagg_xgb * usd_fixer
# sample_submission.to_csv(f'submission_bagXGB_v{VERSION}_currency_correct.csv', index=False)
# Kaggle MAPE 15.61873

# stacking and ensembling
# predict_ensemble = np.exp(st_ensemble.predict(X_sub))
# sample_submission['price'] = predict_ensemble
# sample_submission.to_csv(f'submission_st_Ens_v{VERSION}.csv', index=False)
# Kaggle MAPE 18.35810

В итоге получили **MAPE 15.18838** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

В целом, по завершении проекта делаю вывод о том, что наибольший прирост в качестве модели получается засчет скармливания новых данных. Больше данных - точнее модель. При этом нужно учитывать количество ресурсов (вычислительных, временных) на сбор и обработку данных, расчет и моделирование.

# TODO

1. Изучть как можно использовать обработку изображений и информации с них для улучшения модели.
2. Deal with 1km-mileage
3. Replace EngineDisplacement=0 with groupped medians of EnginePower
4. LGMBR + GridSearchCV для n_estimators >>> 1000. Bagging for Best-LGMBR.
5. Stacking для LGBMR с параметрами, RandomForrestRegressor с параметрами и XGBoost с параметрами.